In [1]:
# Import libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import glob
import os
import datetime
from datetime import time
from datetime import timedelta
from datetime import datetime

# Import plotly express for EF plot
# import plotly.express as px
# import plotly.graph_objects as go
# px.defaults.width, px.defaults.height = 1000,600
# Set precision
pd.set_option('display.precision', 4)

In [2]:
df4 = pd.read_csv('options_below_sma932.csv', index_col=0)

In [3]:
df4.columns

Index(['quote_datetime', 'strike', 'Open', 'High', 'Low', 'Close', 'day_close',
       'prev_close', 'PDL', 'PDH', 'prev_day_neg', 'range',
       'realized_volatility_post30min', 'realized_volatility_full_day',
       'sma_50', 'sma_21', 'below_sma_50', 'gap_new931', 'intraday_sma_11',
       'intraday_sma_20', 'date1', 'high931', 'low931', 'close931', 'open931',
       'time', 'straddle_exit', 'call_price', 'put_price', 'strike932', 'date',
       'call932', 'put932', 'straddle932', 'spot932'],
      dtype='object')

In [4]:
del df4['intraday_sma_20']

In [5]:
df4['quote_datetime'] = pd.to_datetime(df4['quote_datetime'], infer_datetime_format=True, errors='coerce')

In [6]:
df4['date'] = df4['quote_datetime'].dt.date

In [7]:
df4.tail()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,time,straddle_exit,call_price,put_price,strike932,date,call932,put932,straddle932,spot932
155206,2025-01-06 15:56:00,5985.0,5970.99,5974.15,5970.58,5972.82,5975.38,5942.47,5888.66,5949.34,...,15:56:00,12.175,0.075,12.10,5985.0,2025-01-06,13.85,14.65,28.5,5987.42
155207,2025-01-06 15:57:00,5985.0,5973.04,5974.08,5972.24,5972.61,5975.38,5942.47,5888.66,5949.34,...,15:57:00,12.325,0.075,12.25,5985.0,2025-01-06,13.85,14.65,28.5,5987.42
155208,2025-01-06 15:58:00,5985.0,5972.79,5973.07,5968.84,5969.69,5975.38,5942.47,5888.66,5949.34,...,15:58:00,15.675,0.025,15.65,5985.0,2025-01-06,13.85,14.65,28.5,5987.42
155209,2025-01-06 15:59:00,5985.0,5969.50,5971.01,5968.63,5971.01,5975.38,5942.47,5888.66,5949.34,...,15:59:00,13.375,0.025,13.35,5985.0,2025-01-06,13.85,14.65,28.5,5987.42
155210,2025-01-06 16:00:00,5985.0,5971.15,5976.90,5971.08,5976.50,5975.38,5942.47,5888.66,5949.34,...,16:00:00,10.075,0.025,10.05,5985.0,2025-01-06,13.85,14.65,28.5,5987.42


In [8]:
# df6.to_excel('straddle932.xlsx')

In [9]:
# Specify the columns you want to import
column_headers = ['futures_Date', 'futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']

# Import only the OHLC columns
df_spot = pd.read_csv('ES_full_1min.txt', names=column_headers)

# Ensure the 'Date' column is in datetime format
df_spot['futures_Date'] = pd.to_datetime(df_spot['futures_Date'])

# Sort the DataFrame by the 'Date' column
df_spot = df_spot.sort_values(by='futures_Date').reset_index(drop=True)

# Replace commas and convert columns to float
df_spot[['futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']] = df_spot[['futures_Open', 'futures_High', 'futures_Low', 'futures_Close', 'futures_Volume']].replace({',': ''}, regex=True).astype(float)

In [10]:
# Add 1 minute to every value in the 'Date' column
df_spot['futures_Date_plus_1min'] = df_spot['futures_Date'] + pd.DateOffset(minutes=1)

In [11]:
del df_spot['futures_Date']
del df_spot['futures_Volume']

In [12]:
# Filter rows where the time falls between 9:35 AM and 4:10 PM
start_time = pd.to_datetime('09:31:00').time()
end_time = pd.to_datetime('16:00:00').time()

# Apply the filter based on the time component of 'quote_datetime'
df_spot = df_spot[(df_spot['futures_Date_plus_1min'].dt.time >= start_time) & (df_spot['futures_Date_plus_1min'].dt.time <= end_time)]

In [13]:
df_spot = df_spot.reset_index(drop=True)

In [14]:
df_spot['intraday_sma_20'] = (
    df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date)['futures_Close']
    .transform(lambda x: x.rolling(window=20, min_periods=1).mean())
)

In [15]:
# Group by date and calculate the number of unique times per date
df_spot = df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date).filter(
    lambda x: len(x['futures_Date_plus_1min'].dt.time.unique()) == 390
)

In [16]:
df4.columns

Index(['quote_datetime', 'strike', 'Open', 'High', 'Low', 'Close', 'day_close',
       'prev_close', 'PDL', 'PDH', 'prev_day_neg', 'range',
       'realized_volatility_post30min', 'realized_volatility_full_day',
       'sma_50', 'sma_21', 'below_sma_50', 'gap_new931', 'intraday_sma_11',
       'date1', 'high931', 'low931', 'close931', 'open931', 'time',
       'straddle_exit', 'call_price', 'put_price', 'strike932', 'date',
       'call932', 'put932', 'straddle932', 'spot932'],
      dtype='object')

In [17]:
df_spot['futures_Low_min'] = df_spot.groupby(df_spot['futures_Date_plus_1min'].dt.date)['futures_Low'].cummin()

In [18]:
# df_spot[df_spot['futures_Date_plus_1min'].dt.date == pd.to_datetime('2018-01-16').date()].to_csv('ch.csv')

In [19]:
df = pd.merge(df4, df_spot, left_on =  'quote_datetime', right_on = 'futures_Date_plus_1min', how = 'inner')

In [20]:
df.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2686.50,2687.25,2681.25,2682.25,2018-02-07 09:32:00,2684.3750,2680.75
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2682.25,2689.75,2682.25,2687.50,2018-02-07 09:33:00,2685.4167,2680.75
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2687.25,2689.25,2685.75,2688.75,2018-02-07 09:34:00,2686.2500,2680.75
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2688.75,2689.75,2683.75,2689.75,2018-02-07 09:35:00,2686.9500,2680.75
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2689.50,2694.00,2687.00,2692.75,2018-02-07 09:36:00,2687.9167,2680.75


In [21]:
# Calculate the number of unique days
num_unique_days = df4['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

398


In [22]:
df['date'] = df['quote_datetime'].dt.date

In [23]:
# Identify dates where all `straddle_exit` values are >= 0 for every ticker
valid_dates = df.groupby('date')['straddle_exit'].apply(lambda x: (x >= 0).all())

In [24]:
# Filter `df_atm` to keep only rows from the valid dates
df = df[df['date'].isin(valid_dates.index)]

In [25]:
df['time'] = df['quote_datetime'].dt.time

In [26]:
# #Read the CSV file
# dates = pd.read_excel('E:\marker2nitesh.xlsx')

# #Extract the 'Date' column (replace 'Date' with the actual column name in your CSV)
# dates_to_plot = dates['date'].astype(str).tolist()

In [27]:
# dates_to_plot = pd.to_datetime(dates_to_plot).date
# # Filter the DataFrame for the specified dates
# df = df[df['date'].isin(dates_to_plot)]

In [28]:
# Calculate the number of unique days
num_unique_days = df['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [29]:
df.columns

Index(['quote_datetime', 'strike', 'Open', 'High', 'Low', 'Close', 'day_close',
       'prev_close', 'PDL', 'PDH', 'prev_day_neg', 'range',
       'realized_volatility_post30min', 'realized_volatility_full_day',
       'sma_50', 'sma_21', 'below_sma_50', 'gap_new931', 'intraday_sma_11',
       'date1', 'high931', 'low931', 'close931', 'open931', 'time',
       'straddle_exit', 'call_price', 'put_price', 'strike932', 'date',
       'call932', 'put932', 'straddle932', 'spot932', 'futures_Open',
       'futures_High', 'futures_Low', 'futures_Close',
       'futures_Date_plus_1min', 'intraday_sma_20', 'futures_Low_min'],
      dtype='object')

In [30]:
df = df.reset_index(drop=True)

In [31]:
# df = df[df['date'] == pd.to_datetime('2024-04-16').date()]

In [32]:
# df.head(14)

In [33]:
# def filter_based_on_highs(df):
#     # Initialize an empty DataFrame to store results
#     filtered_df = pd.DataFrame()

#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Sort by time to ensure correct order
#         group = group.sort_values(by='time').reset_index(drop=True)

#         # Identify the highest high of the first three candles
#         first_three_high = group.loc[
#             group['time'].isin([time(9, 32), time(9, 33), time(9, 34)]), 'futures_High'
#         ].max()
#         current_high = first_three_high
#         filter_start_index = None
#         high_updated_once = False  # Flag to track if `current_high` has been updated once

#         # Iterate through the subsequent candles
#         for idx, row in group.iterrows():
#             if row['time'] in [time(9, 32), time(9, 33), time(9, 34)]:
#                 continue  # Skip the first three candles already processed
            
#             if row['futures_Close'] > current_high:
#                 # Mark the index to start filtering from here
#                 filter_start_index = idx
#                 break
#             elif not high_updated_once and row['futures_High'] > current_high:
#                 # Update the highest high only once
#                 current_high = row['futures_High']
#                 high_updated_once = True
        
#         # Filter the DataFrame from the identified starting point
#         if filter_start_index is not None:
#             filtered_df = pd.concat([filtered_df, group.iloc[filter_start_index:]])
    
#     # Reset index for the final DataFrame
#     filtered_df.reset_index(drop=True, inplace=True)
#     return filtered_df

In [34]:
# # Apply the function to your DataFrame
# df5 = filter_based_on_highs(df)

In [35]:
# Calculate the number of unique days
num_unique_days = df['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [36]:
df.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2686.50,2687.25,2681.25,2682.25,2018-02-07 09:32:00,2684.3750,2680.75
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2682.25,2689.75,2682.25,2687.50,2018-02-07 09:33:00,2685.4167,2680.75
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2687.25,2689.25,2685.75,2688.75,2018-02-07 09:34:00,2686.2500,2680.75
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2688.75,2689.75,2683.75,2689.75,2018-02-07 09:35:00,2686.9500,2680.75
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2689.50,2694.00,2687.00,2692.75,2018-02-07 09:36:00,2687.9167,2680.75


In [37]:
# def filter_low_break(df):
#     # Initialize an empty DataFrame to store results
#     filtered_df = pd.DataFrame()

#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Sort by time to ensure correct order
#         group = group.sort_values(by='time').reset_index(drop=True)

#         # Initialize a variable to store the previous low
#         previous_low = None
#         filter_start_index = None

#         # Iterate through the DataFrame
#         for idx, row in group.iterrows():
#             if previous_low is not None and row['futures_Close'] < previous_low:
#                 # Mark the index to start filtering
#                 filter_start_index = idx
#                 break

#             # Update the previous low for the next iteration
#             previous_low = row['futures_Low']

#         # Filter the DataFrame from the identified starting point
#         if filter_start_index is not None:
#             filtered_df = pd.concat([filtered_df, group.iloc[filter_start_index:]])

#     # Reset index for the final DataFrame
#     filtered_df.reset_index(drop=True, inplace=True)
#     return filtered_df

In [38]:
# df6 = filter_low_break(df5)

In [39]:
df.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,put932,straddle932,spot932,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2686.50,2687.25,2681.25,2682.25,2018-02-07 09:32:00,2684.3750,2680.75
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2682.25,2689.75,2682.25,2687.50,2018-02-07 09:33:00,2685.4167,2680.75
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2687.25,2689.25,2685.75,2688.75,2018-02-07 09:34:00,2686.2500,2680.75
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2688.75,2689.75,2683.75,2689.75,2018-02-07 09:35:00,2686.9500,2680.75
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,13.1,30.1,2684.43,2689.50,2694.00,2687.00,2692.75,2018-02-07 09:36:00,2687.9167,2680.75


In [40]:
df6 = df.copy()

In [41]:
# Ensure 'date' is a column or derived appropriately
df6['selling_call_price'] = (
    df6.groupby('date')['call_price']
    .transform('first')
)
df6['selling_call_price'] = df6['selling_call_price'].ffill()

In [42]:
# Ensure 'date' is a column or derived appropriately
df6['selling_put_price'] = (
    df6.groupby('date')['put_price']
    .transform('first')
)
df6['selling_put_price'] = df6['selling_put_price'].ffill()

In [43]:
# Ensure 'date' is a column or derived appropriately
df6['selling_time'] = (
    df6.groupby('date')['time']
    .transform('first')
)
df6['selling_time'] = df6['selling_time'].ffill()

In [44]:
df6.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,futures_Open,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,2686.50,2687.25,2681.25,2682.25,2018-02-07 09:32:00,2684.3750,2680.75,17.0,13.1,09:32:00
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,2682.25,2689.75,2682.25,2687.50,2018-02-07 09:33:00,2685.4167,2680.75,17.0,13.1,09:32:00
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,2687.25,2689.25,2685.75,2688.75,2018-02-07 09:34:00,2686.2500,2680.75,17.0,13.1,09:32:00
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,2688.75,2689.75,2683.75,2689.75,2018-02-07 09:35:00,2686.9500,2680.75,17.0,13.1,09:32:00
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,2689.50,2694.00,2687.00,2692.75,2018-02-07 09:36:00,2687.9167,2680.75,17.0,13.1,09:32:00


In [45]:
# Calculate the number of unique days
num_unique_days = df6['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [46]:
def exit_strategy(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'Close' value for each date (first_close)
        #first_close = group['Close'].iloc[0]
        first_call = group['call_price'].iloc[0]
        
        # Step 2: Calculate the condition and filter the rows accordingly
        condition = group['call_price'] < 1.5 * first_call
        
        # Step 3: Find the first row where the condition fails and filter accordingly
        if condition.any():
            first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
            # Keep rows before the first failure (inclusive of the row where it fails)
            if first_fail_idx:
                filtered_group = group.loc[:first_fail_idx]
            else:
                filtered_group = group  # Keep the entire group if the condition never fails
            
            filtered_df_list.append(filtered_group)
    
    return pd.concat(filtered_df_list, ignore_index=True)

In [47]:
# Step 5: Apply the exit strategy
df6 = exit_strategy(df6)

In [48]:
# Calculate the number of unique days
num_unique_days = df6['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [49]:
def exit_strategy1(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_put = group['put_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1.5 * first_put
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        #condition_triggered = False
        #group['put_stop_price'] = np.nan
        group['put_stop_time'] = np.nan
        
        for i in range(len(group)):
            if not stop_loss_triggered and group['put_price'].iloc[i] >= stop_loss_level:
                stop_loss_triggered = True
                a = i
                #stop_loss_time = group['time'].iloc[i] 
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group.loc[group.index[i], 'put_price'] = group.loc[group.index[a], 'put_price']
                #group['stoploss1'] = True
                group['put_stop_time'] = group.loc[group.index[a], 'time']
                #group['stoploss_time'] = stop_loss_time
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [50]:
# Step 5: Apply the exit strategy
df6 = exit_strategy1(df6)

In [51]:
df6[df6['put_stop_time'].notna()].head(8)

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,futures_High,futures_Low,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time,put_stop_time
10,2018-02-09 09:32:00,2610.0,2606.20,2613.07,2605.96,2613.07,2619.55,2581.0,2580.56,2685.27,...,2613.00,2604.75,2612.50,2018-02-09 09:32:00,2608.7500,2604.75,17.85,14.15,09:32:00,09:40:00
11,2018-02-09 09:33:00,2610.0,2613.27,2616.95,2613.27,2616.89,2619.55,2581.0,2580.56,2685.27,...,2615.50,2611.50,2615.25,2018-02-09 09:33:00,2610.9167,2604.75,17.85,14.15,09:32:00,09:40:00
12,2018-02-09 09:34:00,2610.0,2616.95,2617.03,2608.50,2610.21,2619.55,2581.0,2580.56,2685.27,...,2615.75,2606.00,2609.00,2018-02-09 09:34:00,2610.4375,2604.75,17.85,14.15,09:32:00,09:40:00
13,2018-02-09 09:35:00,2610.0,2610.60,2613.42,2610.22,2610.38,2619.55,2581.0,2580.56,2685.27,...,2612.75,2607.50,2608.00,2018-02-09 09:35:00,2609.9500,2604.75,17.85,14.15,09:32:00,09:40:00
14,2018-02-09 09:36:00,2610.0,2609.88,2610.09,2608.15,2608.87,2619.55,2581.0,2580.56,2685.27,...,2608.75,2606.25,2607.00,2018-02-09 09:36:00,2609.4583,2604.75,17.85,14.15,09:32:00,09:40:00
15,2018-02-09 09:37:00,2610.0,2608.82,2608.86,2606.89,2608.28,2619.55,2581.0,2580.56,2685.27,...,2607.75,2605.00,2606.00,2018-02-09 09:37:00,2608.9643,2604.75,17.85,14.15,09:32:00,09:40:00
16,2018-02-09 09:38:00,2610.0,2608.03,2608.03,2603.69,2603.69,2619.55,2581.0,2580.56,2685.27,...,2607.00,2601.25,2601.25,2018-02-09 09:38:00,2608.0000,2601.25,17.85,14.15,09:32:00,09:40:00
17,2018-02-09 09:39:00,2610.0,2603.57,2606.75,2603.33,2605.99,2619.55,2581.0,2580.56,2685.27,...,2606.00,2601.25,2605.25,2018-02-09 09:39:00,2607.6944,2601.25,17.85,14.15,09:32:00,09:40:00


In [52]:
# def exit_strategy2(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):

#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue
            
#         # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
#         first_call = group['call_price'].iloc[0]
        
#         # Step 2: Define the stop-loss level
#         stop_loss_level = 0.3 * first_call

#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         # Step 3: Apply the stop-loss logic
#         stop_loss_triggered = False  # Track if the stop-loss condition has been hit
#         #condition_triggered = False
#         group['call_decay_price'] = np.nan
#         group['call_decay_time'] = np.nan
        
#         for i in range(len(subset)):
#             if not stop_loss_triggered and subset['call_price'].iloc[i] < stop_loss_level:
#                 stop_loss_triggered = True
#                 a = i
#                 #stop_loss_time = group['time'].iloc[i] 
            
#             # Forward-fill the stop-loss level if triggered
#             if stop_loss_triggered:
#                 group['call_decay_price'] = subset.loc[subset.index[a], 'call_price']
#                 #group['stoploss1'] = True
#                 group['call_decay_time'] = subset.loc[subset.index[a], 'time']
#                 #group['stoploss_time'] = stop_loss_time
        
#         # Step 4: Add the adjusted group to the filtered list
#         filtered_df_list.append(group)
    
#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [53]:
def exit_strategy2(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):

        # If all 'put_stop_time' values are NaN, keep the entire group
        if group['put_stop_time'].isna().all():
            filtered_df_list.append(group)
            continue
        
        first_call = group['call_price'].iloc[0]
        
        # Define stop-loss and upper bound
        stop_loss_level = 0.3 * first_call
        upper_bound = 1 * first_call  # Greater than first_call

        # Subset after put_stop_time
        subset = group[group['time'] > group['put_stop_time'].iloc[0]]

        # Track if stop-loss or upper bound condition is hit
        stop_triggered = False  
        
        # Initialize new columns
        group['call_decay_price'] = np.nan
        group['call_decay_time'] = np.nan

        for i in range(len(subset)):
            call_price_i = subset['call_price'].iloc[i]

            # Trigger if call_price < stop_loss_level or call_price > upper_bound
            if not stop_triggered and (call_price_i < stop_loss_level or call_price_i > upper_bound):
                stop_triggered = True
                stop_idx = subset.index[i]
                
            # Forward-fill the stop-loss/upper-bound condition
            if stop_triggered:
                group['call_decay_price'] = group.loc[stop_idx, 'call_price']
                group['call_decay_time'] = group.loc[stop_idx, 'time']

        filtered_df_list.append(group)
    
    return pd.concat(filtered_df_list, ignore_index=True)

In [54]:
# Step 5: Apply the exit strategy
df7 = exit_strategy2(df6)

In [55]:
df7.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,futures_Close,futures_Date_plus_1min,intraday_sma_20,futures_Low_min,selling_call_price,selling_put_price,selling_time,put_stop_time,call_decay_price,call_decay_time
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,2682.25,2018-02-07 09:32:00,2684.3750,2680.75,17.0,13.1,09:32:00,NaN,NaN,NaN
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,2687.50,2018-02-07 09:33:00,2685.4167,2680.75,17.0,13.1,09:32:00,NaN,NaN,NaN
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,2688.75,2018-02-07 09:34:00,2686.2500,2680.75,17.0,13.1,09:32:00,NaN,NaN,NaN
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,2689.75,2018-02-07 09:35:00,2686.9500,2680.75,17.0,13.1,09:32:00,NaN,NaN,NaN
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,2692.75,2018-02-07 09:36:00,2687.9167,2680.75,17.0,13.1,09:32:00,NaN,NaN,NaN


In [56]:
# def exit_strategy3(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):

#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue

#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         # Step 3: Apply the stop-loss logic
#         #condition_triggered = False
#         group['call_condition_price'] = np.nan
#         group['call_firstlow_time'] = np.nan
#         group['call_firstcrossover_time'] = np.nan
#         group['call_secondlow_time'] = np.nan
#         group['call_secondcrossover_time'] = np.nan
    
#         min_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_firstlow_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_firstlow_time'].iloc[0]]
#         min_indices = subset[subset['futures_Close'] > subset['intraday_sma_20']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_firstcrossover_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_firstcrossover_time'].iloc[0]]
#         min_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_secondlow_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         subset = group[group['time'] > group['call_secondlow_time'].iloc[0]]
#         min_indices = subset[subset['futures_Close'] > subset['intraday_sma_20']].groupby(subset['futures_Date_plus_1min'].dt.date).apply(lambda x: x.index.min())
#         group['call_secondcrossover_time'] = subset.loc[min_indices, 'time'].iloc[0]
#         group['call_condition_price'] = subset.loc[min_indices, 'call_price'].iloc[0]        
        
#         # Step 4: Add the adjusted group to the filtered list
#         filtered_df_list.append(group)
    
#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [57]:
def exit_strategy3(df):
    filtered_df_list = []

    # Group by each date
    for date, group in df.groupby('date'):

        # If all 'put_stop_time' values are NaN, skip the group
        if group['put_stop_time'].isna().all():
            filtered_df_list.append(group)  # Append the entire group unchanged
            continue

        # Initialize new columns with NaN
        group['call_condition_price'] = np.nan
        group['call_firstlow_time'] = np.nan
        group['call_firstcrossover_time'] = np.nan
        group['call_secondlow_time'] = np.nan
        group['call_secondcrossover_time'] = np.nan

        # Filter rows where time is after the first 'put_stop_time'
        put_stop_time = group['put_stop_time'].iloc[0]
        subset = group[group['time'] > put_stop_time]

        # Step 1: First low after put stop time
        first_low_indices = subset[subset['futures_Low'] == subset['futures_Low_min']].index
        if not first_low_indices.empty:
            first_low_time = subset.loc[first_low_indices[0], 'time']
            group['call_firstlow_time'] = first_low_time
        else:
            filtered_df_list.append(group)
            continue

        # Step 2: First crossover after the first low
        subset_after_first_low = subset[subset['time'] > first_low_time]
        first_crossover_indices = subset_after_first_low[subset_after_first_low['futures_Close'] > subset_after_first_low['intraday_sma_20']].index
        if not first_crossover_indices.empty:
            first_crossover_time = subset_after_first_low.loc[first_crossover_indices[0], 'time']
            group['call_firstcrossover_time'] = first_crossover_time
        else:
            filtered_df_list.append(group)
            continue

        # Step 3: Second low after the first crossover
        subset_after_first_crossover = subset_after_first_low[subset_after_first_low['time'] > first_crossover_time]
        second_low_indices = subset_after_first_crossover[subset_after_first_crossover['futures_Low'] == subset_after_first_crossover['futures_Low_min']].index
        if not second_low_indices.empty:
            second_low_time = subset_after_first_crossover.loc[second_low_indices[0], 'time']
            group['call_secondlow_time'] = second_low_time
            #group['call_condition_price'] = subset_after_first_crossover.loc[second_low_indices[0], 'call_price']
        else:
            filtered_df_list.append(group)
            continue

        # Step 4: Second crossover after the second low
        subset_after_second_low = subset_after_first_crossover[subset_after_first_crossover['time'] > second_low_time]
        second_crossover_indices = subset_after_second_low[subset_after_second_low['futures_Close'] > subset_after_second_low['intraday_sma_20']].index
        if not second_crossover_indices.empty:
            second_crossover_time = subset_after_second_low.loc[second_crossover_indices[0], 'time']
            group['call_secondcrossover_time'] = second_crossover_time

            # Call price at second crossover
            group['call_condition_price'] = subset_after_second_low.loc[second_crossover_indices[0], 'call_price']

        # Append the processed group
        filtered_df_list.append(group)

    # Combine all filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [58]:
# Step 5: Apply the exit strategy
df8 = exit_strategy3(df7)

In [59]:
df8.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,selling_put_price,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,13.1,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,13.1,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,13.1,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,13.1,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,13.1,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
def calculate_call_final_daily(df):
    # Initialize `call_final` column with NaN
    df['call_final'] = np.nan

    # Group by date and apply the logic
    for date, group in df.groupby('date'):
        call_decay_time = group['call_decay_time'].iloc[0]  # Assuming one value for the day
        call_secondcrossover_time = group['call_secondcrossover_time'].iloc[0]  # Assuming one value for the day

        if pd.isna(call_decay_time) and pd.isna(call_secondcrossover_time):
            # Both are NaN; use call_price for all rows
            df.loc[group.index,'call_final'] = group['call_price']
        elif pd.isna(call_decay_time):
            # Only call_decay_time is NaN; use call_condition_price
            df.loc[group.index, 'call_final'] = group['call_condition_price']
        elif pd.isna(call_secondcrossover_time):
            # Only call_secondcrossover_time is NaN; use call_decay_price
            df.loc[group.index, 'call_final'] = group['call_decay_price']
        else:
            # Both values are non-NaN
            if call_decay_time < call_secondcrossover_time:
                df.loc[group.index, 'call_final'] = group['call_decay_price']
            else:
                df.loc[group.index, 'call_final'] = group['call_condition_price']

    # Fill any remaining NaN values with call_price as a fallback
    #df['call_final'] = df['call_final'].fillna(df['call_price'])
    return df

In [61]:
# Apply the function to df8
df8 = calculate_call_final_daily(df8)

In [62]:
# Calculate the number of unique days
num_unique_days = df8['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [63]:
df8.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-02-07 09:32:00,2680.0,2689.32,2689.32,2684.43,2684.43,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
1,2018-02-07 09:33:00,2680.0,2684.61,2691.16,2684.61,2690.16,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.9
2,2018-02-07 09:34:00,2680.0,2690.08,2690.90,2688.56,2690.48,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.8
3,2018-02-07 09:35:00,2680.0,2690.57,2690.69,2686.65,2690.69,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.8
4,2018-02-07 09:36:00,2680.0,2691.11,2695.17,2690.02,2694.69,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.9


In [64]:
# def exit_strategy3(df):
#     filtered_df_list = []

#     # Group by each date
#     for date, group in df.groupby('date'):
        
#         # If all 'put_stop_time' values are NaN, skip the group
#         if group['put_stop_time'].isna().all():
#             filtered_df_list.append(group)  # Append the entire group
#             continue
            
#         # Step 1: Get the first 'selling_call_price' value for the date (first_call)
#         first_call = group['selling_call_price'].iloc[0]

#         # Create columns for stop values 20, 30, 40, 50, 60, 70
#         stop_levels = [1.2, 1.3, 1.4, 1.5, 1.6, 1.7]

#         subset = group[group['time'] > group['put_stop_time'].iloc[0]]
        
#         for stop in stop_levels:
#             stop_loss_level = stop * first_call
#             stop_loss_triggered = False
#             group[f'call_stop_price_{int(stop * 100)}'] = np.nan
            
#             for i in range(len(subset)):
#                 if not stop_loss_triggered and subset['call_price'].iloc[i] >= stop_loss_level:
#                     stop_loss_triggered = True
#                     a = i

#                 # Forward-fill the stop-loss level if triggered
#                 if stop_loss_triggered:
#                     group[f'call_stop_price_{int(stop * 100)}'] = subset.loc[subset.index[a], 'call_price']

#         # Append the adjusted group to the filtered list
#         filtered_df_list.append(group)

#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [65]:
# def exit_strategy3(df):
#     filtered_df_list = []

#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Skip days where 'put_stop_time' is NaN
#         if group['put_stop_time'].isna().all():
#             continue

#         # Step 1: Get the first 'call_price' value for each date
#         first_call = group['call_price'].iloc[0]

#         # Step 2: Calculate the condition and filter the rows accordingly
#         condition = group['call_price'] < 1.2 * first_call

#         # Step 3: Find the first row where the condition fails and filter accordingly
#         if condition.any():
#             first_fail_idx = condition[~condition].index[0] if (~condition).any() else None

#             # Keep rows before the first failure (inclusive of the row where it fails)
#             if first_fail_idx:
#                 filtered_group = group.loc[:first_fail_idx]
#             else:
#                 filtered_group = group  # Keep the entire group if the condition never fails

#             filtered_df_list.append(filtered_group)

#     return pd.concat(filtered_df_list, ignore_index=True)

In [66]:
# # Step 5: Apply the exit strategy
# df7 = exit_strategy3(df7)

In [67]:
df8 = df8.groupby('date').tail(1).reset_index(drop=True)

In [68]:
df8.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-02-07 09:41:00,2680.0,2698.90,2706.87,2698.90,2706.87,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.80
1,2018-02-09 16:00:00,2610.0,2615.75,2619.64,2612.31,2618.43,2619.55,2581.00,2580.56,2685.27,...,09:32:00,09:40:00,20.05,09:48:00,6.45,09:41:00,09:44:00,10:40:00,10:55:00,20.05
2,2018-02-12 13:55:00,2645.0,2663.52,2665.05,2663.29,2665.05,2656.00,2619.55,2532.69,2638.67,...,09:32:00,10:21:00,15.00,12:10:00,NaN,10:24:00,10:35:00,NaN,NaN,15.00
3,2018-02-14 10:11:00,2650.0,2664.77,2670.22,2664.39,2668.48,2698.63,2662.94,2637.08,2668.84,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.95
4,2018-02-21 09:36:00,2720.0,2724.18,2727.10,2724.18,2727.02,2701.33,2716.26,2706.76,2737.60,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.20


In [69]:
df8.columns

Index(['quote_datetime', 'strike', 'Open', 'High', 'Low', 'Close', 'day_close',
       'prev_close', 'PDL', 'PDH', 'prev_day_neg', 'range',
       'realized_volatility_post30min', 'realized_volatility_full_day',
       'sma_50', 'sma_21', 'below_sma_50', 'gap_new931', 'intraday_sma_11',
       'date1', 'high931', 'low931', 'close931', 'open931', 'time',
       'straddle_exit', 'call_price', 'put_price', 'strike932', 'date',
       'call932', 'put932', 'straddle932', 'spot932', 'futures_Open',
       'futures_High', 'futures_Low', 'futures_Close',
       'futures_Date_plus_1min', 'intraday_sma_20', 'futures_Low_min',
       'selling_call_price', 'selling_put_price', 'selling_time',
       'put_stop_time', 'call_decay_price', 'call_decay_time',
       'call_condition_price', 'call_firstlow_time',
       'call_firstcrossover_time', 'call_secondlow_time',
       'call_secondcrossover_time', 'call_final'],
      dtype='object')

In [70]:
# Calculate the number of unique days
num_unique_days = df8['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [71]:
# decay_levels = [0.3, 0.4, 0.5]

# for decay in decay_levels:
#     df7[f'call_pnl_{int(decay * 100)}'] = df7.apply(
#         lambda row: (
#             row['selling_call_price'] - row['call_price']
#         ) if pd.isna(row[f'call_decay_price_{int(decay * 100)}']) else (
#             row['selling_call_price'] - row[f'call_decay_price_{int(decay * 100)}']
#         ),
#         axis=1
#     )

In [72]:
df8.head()

,quote_datetime,strike,Open,High,Low,Close,day_close,prev_close,PDL,PDH,...,selling_time,put_stop_time,call_decay_price,call_decay_time,call_condition_price,call_firstlow_time,call_firstcrossover_time,call_secondlow_time,call_secondcrossover_time,call_final
0,2018-02-07 09:41:00,2680.0,2698.90,2706.87,2698.90,2706.87,2681.66,2695.14,2593.07,2701.04,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.80
1,2018-02-09 16:00:00,2610.0,2615.75,2619.64,2612.31,2618.43,2619.55,2581.00,2580.56,2685.27,...,09:32:00,09:40:00,20.05,09:48:00,6.45,09:41:00,09:44:00,10:40:00,10:55:00,20.05
2,2018-02-12 13:55:00,2645.0,2663.52,2665.05,2663.29,2665.05,2656.00,2619.55,2532.69,2638.67,...,09:32:00,10:21:00,15.00,12:10:00,NaN,10:24:00,10:35:00,NaN,NaN,15.00
3,2018-02-14 10:11:00,2650.0,2664.77,2670.22,2664.39,2668.48,2698.63,2662.94,2637.08,2668.84,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.95
4,2018-02-21 09:36:00,2720.0,2724.18,2727.10,2724.18,2727.02,2701.33,2716.26,2706.76,2737.60,...,09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.20


In [73]:
# Update 'call_price' based on the condition
df8['call_final'] = np.where(
    (df8['time'] == pd.Timestamp('16:00').time()) & ~df8['day_close'].isna(),
    np.maximum(df8['day_close'] - df8['strike'], 0),
    df8['call_final']  # Set to NaN if 'time' is not 16:00 or 'day_close' is NaN
)

In [74]:
df8['put_price'] = np.where(
    (df8['time'] == pd.Timestamp('16:00').time()) & ~df8['day_close'].isna(),
    np.maximum(-df8['day_close'] + df8['strike'], 0),
    df8['put_price']  # Set to NaN if 'time' is not 16:00 or 'day_close' is NaN
)

In [75]:
df8['call_pnl'] = df8['selling_call_price'] - df8['call_final']
df8['put_pnl'] = df8['selling_put_price'] - df8['put_price']

In [76]:
df8 = df8.sort_values(by='quote_datetime').reset_index(drop=True)

In [77]:
df8 = df8[df8['call_price'] >= 0]

In [78]:
df8 = df8[df8['put_price'] >= 0]

In [79]:
# df_filtered1['time'] = df_filtered1['quote_datetime'].dt.time

In [80]:
# df8['straddle_pnl_bps'] =  (df8['straddle_pnl']/ df8['spot932'])*10000
df8['put_pnl_bps'] =  (df8['put_pnl']/ df8['futures_Close'])*10000
df8['call_pnl_bps'] =  (df8['call_pnl']/ df8['futures_Close'])*10000

# for stop in stop_levels:
#     df8[f'call_stop_pnl_bps_{int(stop * 100)}'] = (df8[f'call_stop_pnl_{int(stop * 100)}']/ df8['futures_Close'])*10000

In [81]:
# df5 = df8[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'Close', 'spot932', 'gap931', 'realized_volatility_post30min', 'prev_close', 'PDL', 'PDH', 'gap', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'intraday_sma_11', 'below_sma_50', 'day_close', 'straddle932', 'call932', 'call_price', 'call_pnl', 'call_pnl_bps', 'put932', 'put_stop1_price', 'put_price', 'put_stoploss_time1', 'put_pnl', 'put_pnl_bps']]

In [82]:
df9 = df8[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'futures_Close', 'gap_new931', 'spot932', 'realized_volatility_post30min', 'day_close', 'PDH', 'PDL', 'prev_close', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50', 'day_close', 'straddle932', 'straddle_exit', 'selling_time', 'put932', 'selling_put_price', 'put_price', 'put_stop_time', 'put_pnl', 'put_pnl_bps', 'call932', 'selling_call_price', 'call_price', 'call_decay_price', 'call_decay_time',
       'call_condition_price', 'call_firstlow_time', 'call_firstcrossover_time', 'call_secondlow_time', 'call_secondcrossover_time', 'call_final', 'call_pnl', 'call_pnl_bps']]

In [83]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [84]:
df9['year'] = df9['quote_datetime'].dt.year

In [85]:
# df6 = df5.groupby('date').tail(1).reset_index(drop=True)

In [86]:
df9 = df9[df9['below_sma_50'] == 'Yes']

In [87]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [88]:
# df5['total_pnl_bps'] = df5['call_pnl_bps'] + df5['put_pnl_bps']

In [89]:
# df9 = df9[(df9['below_sma_50'] == 'Yes') & (df9['prev_day_neg'] == 'yes')]

In [90]:
# df9 = df9[df9['gap']/df9['straddle932'] < 1]

In [91]:
# Calculate the number of unique days
num_unique_days = df9['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [92]:
df10 = df9.copy()

In [93]:
# # Convert '10:32' to a datetime.time object
# time_limit = datetime.strptime('10:32', '%H:%M').time()

# # Filter the DataFrame
# df10 = df9[df9['selling_time'] <= time_limit]

In [94]:
# Calculate the number of unique days
num_unique_days = df10['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [95]:
df10['final_pnl_bps'] = df10['call_pnl_bps'] + df10['put_pnl_bps']/2

In [96]:
# df10['final_pnl_bps120'] = df10['call_stop_pnl_bps_120'] + df10['put_pnl_bps']
# df10['final_pnl_bps130'] = df10['call_stop_pnl_bps_130'] + df10['put_pnl_bps']
# df10['final_pnl_bps140'] = df10['call_stop_pnl_bps_140'] + df10['put_pnl_bps']
# df10['final_pnl_bps150'] = df10['call_stop_pnl_bps_150'] + df10['put_pnl_bps']
# df10['final_pnl_bps160'] = df10['call_stop_pnl_bps_160'] + df10['put_pnl_bps']
# df10['final_pnl_bps170'] = df10['call_stop_pnl_bps_170'] + df10['put_pnl_bps']

In [97]:
# Initialize an empty DataFrame to store yearly drawdown information
yearly_drawdowns = []

# Group the DataFrame by year and calculate the drawdowns
for year, group in df10.groupby('year'):
    # Calculate the cumulative sum of pnl_bps
    group['cumulative_pnl'] = (group['final_pnl_bps']).cumsum()
    
    # Calculate the running maximum of the cumulative pnl
    group['running_max'] = group['cumulative_pnl'].cummax()
    
    # Calculate the drawdown as the difference between the running max and the current cumulative pnl
    group['drawdown'] = group['running_max'] - group['cumulative_pnl']
    
    # Find the maximum drawdown for the year
    max_drawdown = group['drawdown'].max()
    
    # Append the results to the list
    yearly_drawdowns.append({'year': year, 'max_drawdown': max_drawdown})

# Create a DataFrame from the yearly drawdown information
yearly_drawdowns_df = pd.DataFrame(yearly_drawdowns)

print(yearly_drawdowns_df)

   year  max_drawdown
0  2018      335.5387
1  2019      105.6347
2  2020      136.1743
3  2021       51.7424
4  2022      253.4754
5  2023      145.1594
6  2024       84.6048


In [98]:
df10 = df10[df10['selling_time'] <= pd.to_datetime('10:32').time()]

In [99]:
# Calculate the number of unique days
num_unique_days = df10['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

391


In [100]:
df10 = df10[df10['prev_day_neg'] == 'yes']

In [101]:
# Calculate the number of unique days
num_unique_days = df10['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

223


In [102]:
df10.to_excel('fut932_50call_50put_30decaytrail.xlsx')